In [1]:
# Import required libraries for YouTube transcript extraction
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled

# Test the import
print("YouTube Transcript API imported successfully!")
print("Available methods:", [method for method in dir(YouTubeTranscriptApi) if not method.startswith('_')])

YouTube Transcript API imported successfully!
Available methods: ['fetch', 'list']


## Step 1a Indexing (Document Ingestion)

In [2]:
# Extract YouTube transcript with timing information
video_id = "Gfr50f6ZBvo"

try:
    api = YouTubeTranscriptApi()
    transcript_list = api.fetch(video_id, languages=["en"])
    structured_transcript = []
    for snippet in transcript_list:
        transcript_entry = {
            'text': snippet.text,
            'start': snippet.start,
            'duration': snippet.duration
        }
        structured_transcript.append(transcript_entry)
except Exception as e:
    transcript_data = YouTubeTranscriptApi.get_transcript(video_id)
    structured_transcript = [{'text': item['text'], 'start': item['start'], 'duration': item['duration']} for item in transcript_data]

# Display transcript in the requested format
for segment in structured_transcript:
    print(f"  {{'text': '{segment['text']}',")
    print(f"   'start': {segment['start']},")
    print(f"   'duration': {segment['duration']}}},")

  {'text': 'the following is a conversation with',
   'start': 0.08,
   'duration': 3.44},
  {'text': 'demus hasabis',
   'start': 1.76,
   'duration': 4.96},
  {'text': 'ceo and co-founder of deepmind',
   'start': 3.52,
   'duration': 5.119},
  {'text': 'a company that has published and builds',
   'start': 6.72,
   'duration': 4.48},
  {'text': 'some of the most incredible artificial',
   'start': 8.639,
   'duration': 4.561},
  {'text': 'intelligence systems in the history of',
   'start': 11.2,
   'duration': 4.8},
  {'text': 'computing including alfred zero that',
   'start': 13.2,
   'duration': 3.68},
  {'text': 'learned',
   'start': 16.0,
   'duration': 2.96},
  {'text': 'all by itself to play the game of gold',
   'start': 16.88,
   'duration': 4.559},
  {'text': 'better than any human in the world and',
   'start': 18.96,
   'duration': 5.6},
  {'text': 'alpha fold two that solved protein',
   'start': 21.439,
   'duration': 4.241},
  {'text': 'folding',
   'start': 24.56,


In [3]:
# Flatten it to plain text
transcript = " ".join(chunk["text"] for chunk in structured_transcript)
print(transcript)

the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough 

## step-1b Indexing(text splitting)

In [4]:
# Import text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split the transcript into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])


In [5]:
len(chunks)

168

In [6]:
chunks[100]

Document(metadata={}, page_content="and and kind of come up with descriptions of the electron clouds where they're gonna go how they're gonna interact when you put two elements together uh and what we try to do is learn a simulation uh uh learner functional that will describe more chemistry types of chemistry so um until now you know you can run expensive simulations but then you can only simulate very small uh molecules very simple molecules we would like to simulate large materials um and so uh today there's no way of doing that and we're building up towards uh building functionals that approximate schrodinger's equation and then allow you to describe uh what the electrons are doing and all materials sort of science and material properties are governed by the electrons and and how they interact so have a good summarization of the simulation through the functional um but one that is still close to what the actual simulation would come out with so what um how difficult is that to ask w

## Step 1c and 1d Indexing(Embedding Generation and storing in vector store)

In [7]:
# Import Hugging Face embeddings
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize Hugging Face embeddings model
# Using a popular sentence transformer model that's great for RAG
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU
)

print("✅ Hugging Face embeddings model loaded successfully!")
print(f"Model: sentence-transformers/all-MiniLM-L6-v2")
print(f"Embedding dimension: 384")

/home/ganesh/Desktop/LanChain-Framework/langchain-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Hugging Face embeddings model loaded successfully!
Model: sentence-transformers/all-MiniLM-L6-v2
Embedding dimension: 384


In [8]:
# Import FAISS vector store
from langchain_community.vectorstores import FAISS

# Create vector store from documents and embeddings
print("🔄 Creating FAISS vector store...")
print(f"📄 Processing {len(chunks)} document chunks...")

vector_store = FAISS.from_documents(chunks, embeddings)

print("✅ FAISS vector store created successfully!")
print(f"📊 Vector store contains {vector_store.index.ntotal} vectors")
print(f"🔍 Ready for similarity search!")

🔄 Creating FAISS vector store...
📄 Processing 168 document chunks...
✅ FAISS vector store created successfully!
📊 Vector store contains 168 vectors
🔍 Ready for similarity search!


In [9]:
vector_store.index_to_docstore_id

{0: '62d90e28-7550-4da4-9d32-a69892d71224',
 1: 'a0db17eb-9b12-4831-88e6-596892b1c936',
 2: 'cbf4679a-bd51-4403-9bae-e91ad46a2042',
 3: '53029be0-49d3-42c2-8e40-678588129413',
 4: '5beb7bd0-50d6-4895-8981-9aacde3ad0b9',
 5: 'd75bb367-2d3f-4920-b348-cb06e3c55700',
 6: 'b6222a82-974d-4231-bcc1-87cd5a565737',
 7: '92ecca72-6667-4b25-a822-a1b135ec44c3',
 8: 'a9ae2f84-b5ad-43f0-95ae-3d33b786011a',
 9: '74d74867-70b0-46c2-acab-b58827a8df3a',
 10: 'decb4182-9f90-42ef-90a4-d2ff7796d4e5',
 11: '0137a4f4-6f89-4f3d-94cc-549d1e425426',
 12: '6eb07a26-ffea-4851-930f-f11b7edf087a',
 13: '5e142777-f6de-4b8a-a197-25a340168870',
 14: '7a9cc5a2-a922-4614-b67f-f9a4ffc6fc8f',
 15: '225c7eeb-98b7-4190-9997-173f3cedda36',
 16: '8baa5ee4-dfc1-4ea1-bdf4-825c4a2856de',
 17: '71e47000-ae96-4909-b4b7-df2365061aa0',
 18: 'a57e28bf-470a-4153-94c6-b26a58e22d9f',
 19: '29595f40-2ab4-456c-b258-4a00d311f5d1',
 20: '3e623512-5674-4ce4-8e44-4f5d5f99300b',
 21: 'accc8cef-3340-4918-8483-13410f3e1b3f',
 22: 'ea2ae0e6-201e-

In [10]:
vector_store.get_by_ids(['aa47cef7-0a1b-44fc-a7f7-d867260fc02d'])

[]

## Step 2 -Retriveal

In [11]:
retriver = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [12]:
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x723653d1bfb0>, search_kwargs={'k': 4})

In [13]:
retriver.invoke('what is deepmind?')

[Document(id='d4a63f03-3613-45f1-b65b-d977624e154d', metadata={}, page_content="and how it works this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um uh if if you were to sort of look back like if we go forward 200 years look back what was the key 

## Step 3- Augmentation

In [14]:
# Import Hugging Face LLM
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize Hugging Face LLM using your API token
llm_endpoint = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",  # Good for Q&A tasks
    task="text-generation",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN"),
    temperature=0.2,
    max_new_tokens=512
)

llm = ChatHuggingFace(llm=llm_endpoint)

print("✅ Hugging Face LLM loaded successfully!")
print("📋 Model: meta-llama/Llama-3.1-8B-Instruct")
print("🎯 Ready for RAG question answering!")

✅ Hugging Face LLM loaded successfully!
📋 Model: meta-llama/Llama-3.1-8B-Instruct
🎯 Ready for RAG question answering!


In [15]:
# Import PromptTemplate from the correct module
from langchain_core.prompts import PromptTemplate

# Create prompt template for RAG
prompt = PromptTemplate(
    template="""You are a helpful assistant.
Answer ONLY from the provided transcript context.
If the context is insufficient, just say you don't know.

Context: {context}

Question: {question}

Answer:""",
    input_variables=['context', 'question']
)

print("✅ Prompt template created successfully!")
print("📋 Template ready for RAG question answering")

✅ Prompt template created successfully!
📋 Template ready for RAG question answering


In [16]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriver.invoke(question)

In [17]:
retrieved_docs 

[Document(id='c7cf51d7-acc4-4a07-be49-00ef7e927687', metadata={}, page_content="in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be intere

In [18]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of\n\

In [19]:
final_prompt = prompt.invoke({"context":context_text, "question":question})

In [20]:
final_prompt

StringPromptValue(text="You are a helpful assistant.\nAnswer ONLY from the provided transcript context.\nIf the context is insufficient, just say you don't know.\n\nContext: in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones 

## Step 4 - Generation

In [21]:
answer = llm.invoke(final_prompt)
print(answer.content)

The topic of nuclear fusion is discussed in this video.

Nuclear fusion is discussed multiple times, with a mention of the test reactor in Switzerland that the team collaborated with EPFL (École polytechnique fédérale de Lausanne) and a Swiss technical institute to use. The topic is also discussed in relation to the challenges and bottlenecks in making fusion work, and how the team's AI methods can help with those challenges. 

Additionally, there is a specific mention of the work on magnetic control of tokamak plasmas to deep reinforcement learning, which is a part of the team's research in nuclear fusion.


# Building a chain

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [23]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [24]:
parallel_chain = RunnableParallel({
    'context' : retriver | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [25]:
parallel_chain.invoke('who is Demis')

{'context': "to get world peace because there's also other corrupting things like wanting power over people and this kind of stuff which is not necessarily satisfied by by just abundance but i think it will help um and i think uh but i think ultimately ai is not going to be run by any one person or one organization i think it should belong to the world belong to humanity um and i think maybe many there'll be many ways this will happen and ultimately um everybody should have a say in that do you have advice for uh young people in high school and college maybe um if they're interested in ai or interested in having a big impact on the world what they should do to have a career they can be proud of her to have a life they can be proud of i love giving talks to the next generation what i say to them is actually two things i i think the most important things to learn about and to find out about when you're when you're young is what are your true passions is first of all there's two things on

In [26]:
parser = StrOutputParser()

In [27]:
main_chain = parallel_chain | prompt | llm | parser

In [28]:
main_chain.invoke("who is Demis?")

'Demis Hassabis, CEO and co-founder of DeepMind, a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing.'